In [1]:
# ============================================================
# Comparison Experiment: LLM Interaction without DES Controller (Baseline)
# ============================================================
# This code demonstrates potential violation behaviors of LLM without formal control
# Compare with the version using DES controller in llm_control_tct.ipynb

print("="*60)
print("--- Comparison Experiment: LLM Interaction without Controller (Baseline) ---")
print("--- Scenario: Technical Support Bot ---")
print("="*60)
print("\nNote: This version does not use a DES controller, LLM can freely choose any action")
print("      including behaviors that may violate specifications (e.g., escalating before gathering information)\n")

--- Comparison Experiment: LLM Interaction without Controller (Baseline) ---
--- Scenario: Technical Support Bot ---

Note: This version does not use a DES controller, LLM can freely choose any action
      including behaviors that may violate specifications (e.g., escalating before gathering information)



In [2]:
# --- Define states and events (same Plant model as controlled version) ---

# State definitions
states = {
    0: "S0_Idle",
    1: "S1_ProblemReceived",
    2: "S2_InfoGathering",
    3: "S3_SolutionProposed",
    4: "S4_Escalated",
    5: "S5_Resolved"
}

# All possible transitions (including violating transitions)
transitions = [
    (0, 'p_ask_problem', 1, 'c'),
    (1, 'p_ask_details', 2, 'c'),
    (1, 'r_is_confused', 1, 'u'),
    (2, 'r_provides_info', 3, 'u'),
    (2, 'r_is_confused', 2, 'u'),
    (3, 'r_confirms_solved', 5, 'u'),
    (3, 'r_requests_escalation', 4, 'u'),
    (3, 'r_is_confused', 3, 'u'),
    (1, 'p_escalate', 4, 'c'),  # Violating behavior: escalating before gathering information
    (3, 'p_escalate', 4, 'c'),
    (4, 'r_is_confused', 4, 'u'),
    (4, 'p_end_session', 0, 'c'),
    (5, 'r_is_confused', 5, 'u'),
    (5, 'p_end_session', 0, 'c'),
]

# Build transition dictionary for easy lookup
transition_dict = {}
for src, event, dst, ctrl in transitions:
    if src not in transition_dict:
        transition_dict[src] = []
    transition_dict[src].append((event, dst, ctrl))

print("[1] Plant model definition completed (without controller)")
print(f"    Number of states: {len(states)}")
print(f"    Number of transitions: {len(transitions)}")
print("    ⚠️  Contains violating transition: (S1_ProblemReceived, p_escalate, S4_Escalated)")

[1] Plant model definition completed (without controller)
    Number of states: 6
    Number of transitions: 14
    ⚠️  Contains violating transition: (S1_ProblemReceived, p_escalate, S4_Escalated)


In [3]:
# --- Define Prompt texts and mock LLM ---

prompt_texts = {
    'p_ask_problem': "Hello! I'm a support bot. Please describe your problem.",
    'p_ask_details': "Thank you. Can you provide more details?",
    'p_escalate': "I will escalate this issue to a human agent.",
    'p_end_session': "Session is ending. Goodbye."
}

def mock_llm_uncontrolled(prompt, current_state_name):
    """
    Mock LLM response (uncontrolled version)
    Note: Responses here may lead to violating behaviors
    """
    print(f"[LLM Input] Prompt: '{prompt}' (from state: {current_state_name})")
    
    if "describe your problem" in prompt:
        return "My computer won't turn on.", "r_is_confused"
    elif "more details" in prompt:
        return "I've already checked the power cable.", "r_provides_info"
    elif "ending" in prompt or "Goodbye" in prompt:
        return "Thank you!", "r_confirms_solved"
    else:
        return "I am confused.", "r_is_confused"

print("[2] Prompt texts and mock LLM definition completed")

[2] Prompt texts and mock LLM definition completed


In [4]:
# --- Uncontrolled interaction simulation ---

print("\n" + "="*60)
print("--- Starting uncontrolled interaction simulation ---")
print("="*60)

current_state = 0
violation_detected = False
info_gathered = False  # Flag to track if information has been gathered
step_count = 0

while True:
    step_count += 1
    current_state_name = states[current_state]
    
    print(f"\n{'='*60}")
    print(f"Step {step_count}: Current state = {current_state_name} (State {current_state})")
    print(f"{'='*60}")
    
    # Get all possible transitions from current state
    if current_state not in transition_dict:
        print("[Session] Reached terminal state, session ended.")
        break
    
    available_transitions = transition_dict[current_state]
    
    # Filter controllable events (Prompts)
    controllable_actions = [(e, d) for e, d, c in available_transitions if c == 'c' and e in prompt_texts]
    
    if not controllable_actions:
        print("[Info] No executable Prompts in current state, waiting for uncontrollable events...")
        uncontrollable = [(e, d) for e, d, c in available_transitions if c == 'u']
        if not uncontrollable:
            print("[Session] No available transitions, session ended.")
            break
        event, next_state = uncontrollable[0]
        print(f"[Event] Auto-triggering uncontrollable event: {event}")
        current_state = next_state
        continue
    
    # Display all available actions (including violating actions)
    print("\n--- Available actions (Prompts) ---")
    for i, (event, _) in enumerate(controllable_actions):
        warning = ""
        # Detect violating actions
        if event == 'p_escalate' and not info_gathered:
            warning = " ⚠️  [Violating action: escalating without gathering information]"
        print(f"  {i+1}: {prompt_texts[event]} ({event}){warning}")
    
    # Auto-select (simulating uncontrolled decision-making)
    # To demonstrate violating behavior, we deliberately choose violating actions in some cases
    try:
        choice = int(input("\nEnter option number (or press Enter for auto-select): ").strip() or "1") - 1
        if choice < 0 or choice >= len(controllable_actions):
            choice = 0
    except (ValueError, EOFError):
        choice = 0
    
    chosen_event, next_state = controllable_actions[choice]
    
    # Detect violation
    if chosen_event == 'p_escalate' and not info_gathered:
        print(f"\n{'!'*60}")
        print("⚠️  Violation detected!")
        print("    Specification requires: Before event 'r_provides_info' occurs,")
        print("             event 'p_escalate' must not occur")
        print("    Current behavior: Escalating issue without gathering user information")
        print(f"{'!'*60}\n")
        violation_detected = True
    
    print(f"\n[Action] Executing action: {chosen_event}")
    current_state = next_state
    
    # If ending session, exit directly
    if chosen_event == 'p_end_session':
        print("\n[Session] Session ended.")
        break
    
    # Call LLM to get response
    llm_response_text, llm_response_event = mock_llm_uncontrolled(
        prompt_texts[chosen_event], states[current_state]
    )
    print(f"    [LLM Output] Response: '{llm_response_text}' (classified as: '{llm_response_event}')")
    
    # Record if information was gathered
    if llm_response_event == 'r_provides_info':
        info_gathered = True
        print("    ✓ Information gathering completed")
    
    # Update state based on LLM response
    next_state_found = False
    for event, dst, ctrl in transition_dict.get(current_state, []):
        if event == llm_response_event:
            print(f"[Event] Event occurred: {llm_response_event}")
            current_state = dst
            next_state_found = True
            break
    
    if not next_state_found:
        print(f"\n⚠️  Warning: Unexpected event '{llm_response_event}' occurred in state {current_state}")
        print("[Session] Session terminated abnormally.")
        break
    
    # Limit maximum steps to prevent infinite loops
    if step_count >= 10:
        print("\n[Session] Maximum step limit reached, session ended.")
        break

print("\n" + "="*60)
print("--- Simulation ended ---")
print("="*60)

if violation_detected:
    print("\n❌ Result: Violation detected!")
    print("   Without a DES controller, LLM may execute actions that violate specifications.")
else:
    print("\n✓ Result: No violations detected in this interaction.")
    print("  (But this doesn't mean the system is always safe, just that no violations were triggered in this simulation)")


--- Starting uncontrolled interaction simulation ---

Step 1: Current state = S0_Idle (State 0)

--- Available actions (Prompts) ---
  1: Hello! I'm a support bot. Please describe your problem. (p_ask_problem)

[Action] Executing action: p_ask_problem
[LLM Input] Prompt: 'Hello! I'm a support bot. Please describe your problem.' (from state: S1_ProblemReceived)
    [LLM Output] Response: 'My computer won't turn on.' (classified as: 'r_is_confused')
[Event] Event occurred: r_is_confused

Step 2: Current state = S1_ProblemReceived (State 1)

--- Available actions (Prompts) ---
  1: Thank you. Can you provide more details? (p_ask_details)
  2: I will escalate this issue to a human agent. (p_escalate) ⚠️  [Violating action: escalating without gathering information]

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
⚠️  Violation detected!
    Specification requires: Before event 'r_provides_info' occurs,
             event 'p_escalate' must not occur
    Current behavior: Escala

In [5]:
# --- Comparison summary ---

print("\n" + "="*60)
print("--- Comparison Summary: With Controller vs Without Controller ---")
print("="*60)

print("""
┌─────────────────────────────────────────────────────────────┐
│                    Comparison Dimensions                     │
├─────────────────────────────────────────────────────────────┤
│ 1. State Space                                               │
│    - Without controller: All possible states and transitions │
│                          are reachable                       │
│    - With controller: Only specification-compliant states    │
│                       and transitions are reachable          │
│                                                              │
│ 2. Action Selection                                          │
│    - Without controller: LLM can freely choose any          │
│                          controllable event                  │
│    - With controller: Can only choose controllable events   │
│                       allowed by the controller              │
│                                                              │
│ 3. Specification Guarantee                                   │
│    - Without controller: Cannot guarantee specification      │
│                          compliance, may violate             │
│    - With controller: Formally guarantees specification      │
│                       compliance, no violations              │
│                                                              │
│ 4. Key Difference Example                                    │
│    Specification: "p_escalate cannot occur before            │
│                    r_provides_info"                          │
│                                                              │
│    - Without controller: In S1_ProblemReceived state,        │
│                          LLM can choose p_escalate (violates)│
│                                                              │
│    - With controller: In S1_ProblemReceived state,           │
│                       p_escalate is not in available actions │
│                       (prohibited)                           │
│                                                              │
│ 5. Practical Application Value                               │
│    - Without controller: Suitable for exploratory tasks,     │
│                          but not for critical business       │
│    - With controller: Suitable for critical business         │
│                       scenarios requiring strict compliance  │
│                       (e.g., customer service, healthcare,   │
│                        finance)                              │
└─────────────────────────────────────────────────────────────┘
""")

print("\n💡 Key Insight:")
print("   DES controller uses formal methods to dynamically constrain LLM's action space")
print("   at runtime, ensuring LLM behavior always complies with predefined specifications,")
print("   thereby improving system reliability and safety. This is a strong guarantee that")
print("   traditional prompt engineering cannot achieve.")

print("\n📊 Recommended comparison experiment:")
print("   1. Run this file (llm_tct.ipynb) - Observe behavior without controller")
print("   2. Run llm_control_tct.ipynb - Observe behavior with controller")
print("   3. Compare action selection and specification compliance in the same scenario")
print("   4. Pay special attention to: Whether p_escalate is allowed in S1_ProblemReceived state")


--- Comparison Summary: With Controller vs Without Controller ---

┌─────────────────────────────────────────────────────────────┐
│                    Comparison Dimensions                     │
├─────────────────────────────────────────────────────────────┤
│ 1. State Space                                               │
│    - Without controller: All possible states and transitions │
│                          are reachable                       │
│    - With controller: Only specification-compliant states    │
│                       and transitions are reachable          │
│                                                              │
│ 2. Action Selection                                          │
│    - Without controller: LLM can freely choose any          │
│                          controllable event                  │
│    - With controller: Can only choose controllable events   │
│                       allowed by the controller              │
│                         